In [ ]:
import populator

In [ ]:
data_dir = 'data'
# Second item in tuple is max batch size for requests.
exchanges = [('binance', 1000), ('bitmex', 200)]
pairs = ['BTC/USDT', 'ETH/USDT', 'BTC/USD', 'ETH/USD']
tick_size = '1m'
start = '2018-01-01T00:00:00Z'
num_ticks = -1

In [ ]:
populator.populate(data_dir, exchanges, pairs, tick_size, start)

In [ ]:
import pandas

In [ ]:
btc_usdt_path = populator.get_data_path(data_dir, 'binance', 'BTC/USDT', tick_size, start, num_ticks)
btc_usdt_frame = pandas.read_csv(btc_usdt_path, index_col=0)
btc_usdt_frame.index = pandas.to_datetime(btc_usdt_frame.index, unit='ms')

In [ ]:
btc_usdt_frame

In [ ]:
import backtrader as bt

def test_strategy(strategy, data):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)
    data = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data)
    
    starting_cash = 10**6
    cerebro.broker.setcash(starting_cash)
    cerebro.run()
    
    portfolio_value = cerebro.broker.getvalue()
    pnl = portfolio_value - starting_cash
    
    print('Final Portfolio Value: ${}'.format(portfolio_value))
    print('P/L: ${}'.format(pnl))
    
    cerebro.plot(style='candlestick')

In [ ]:
class SimpleRsiStrategy(bt.Strategy):
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=10)
        else:
            if self.rsi > 70:
                self.sell(size=10)

test_strategy(SimpleRsiStrategy)